In [56]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.


In [57]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

tongpython_cat_and_dog_path = kagglehub.dataset_download('tongpython/cat-and-dog')


print('Data source import complete.')


Using Colab cache for faster access to the 'cat-and-dog' dataset.
Data source import complete.


In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Streaming output truncated to the last 5000 lines.
/kaggle/input/cat-and-dog/training_set/training_set/dogs/dog.3244.jpg
/kaggle/input/cat-and-dog/training_set/training_set/dogs/dog.2373.jpg
/kaggle/input/cat-and-dog/training_set/training_set/dogs/dog.3261.jpg
/kaggle/input/cat-and-dog/training_set/training_set/dogs/dog.614.jpg
/kaggle/input/cat-and-dog/training_set/training_set/dogs/dog.3692.jpg
/kaggle/input/cat-and-dog/training_set/training_set/dogs/dog.1490.jpg
/kaggle/input/cat-and-dog/training_set/training_set/dogs/dog.1805.jpg
/kaggle/input/cat-and-dog/training_set/training_set/dogs/dog.1449.jpg
/kaggle/input/cat-and-dog/training_set/training_set/dogs/dog.3278.jpg
/kaggle/input/cat-and-dog/training_set/training_set/dogs/dog.3748.jpg
/kaggle/input/cat-and-dog/training_set/training_set/dogs/dog.2580.jpg
/kaggle/input/cat-and-dog/training_set/training_set/dogs/dog.2586.jpg
/kaggle/input/cat-and-dog/training_set/training_set/dogs/dog.1996.jpg
/kaggle/input/cat-and-dog/training_set/t

In [59]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense ,Conv2D,Flatten,MaxPooling2D,BatchNormalization , Dropout

In [60]:
train_data = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/cat-and-dog/training_set',
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    validation_split=0.2,
    subset='training',
    seed=123
)
val_data = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/cat-and-dog/test_set',
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    validation_split=0.2,
    subset='training',
    seed=123
)


Found 8005 files belonging to 1 classes.
Using 6404 files for training.
Found 2023 files belonging to 1 classes.
Using 1619 files for training.


In [61]:
#Normalize

def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_data = train_data.map(process)
val_data = val_data.map(process)

In [62]:
# create cnn model

model = Sequential()

model.add(Conv2D(32,kernel_size =(3,3),padding='valid',activation='relu',input_shape =(128,128,3) ))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size =(3,3),padding='valid',activation='relu',input_shape =(128,128,3) ))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size =(3,3),padding='valid',activation='relu',input_shape =(128,128,3) ))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

In [63]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,313,857 (12.64 MB)

 Trainable params: 3,313,409 (12.64 MB)

 Non-trainable params: 448 (1.75 KB)

In [64]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [65]:
history=model.fit(train_data,epochs=10,validation_data=val_data)
history

Epoch 1/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 22s 76ms/step - accuracy: 0.9886 - loss: 0.0325 - val_accuracy: 1.0000 - val_loss: 9.0937e-26
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 1.0000 - loss: 1.0962e-11 - val_accuracy: 1.0000 - val_loss: 8.6869e-07
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 1.0000 - loss: 1.9287e-09 - val_accuracy: 1.0000 - val_loss: 6.2698e-10
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 1.0000 - loss: 4.2352e-11 - val_accuracy: 1.0000 - val_loss: 2.0615e-08
Epoch 5/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - accuracy: 1.0000 - loss: 2.2789e-13 - val_accuracy: 1.0000 - val_loss: 1.6324e-08
Epoch 6/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - accuracy: 1.0000 - loss: 2.4139e-10 - val_accuracy: 1.0000 - val_loss: 1.3360e-08
Epoch 7/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 1.0000 - loss: 4.0411e-09 - val_accuracy: 1.0000 - val_loss: 1.1633e-08
Epoch 8/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 9s 